# Recommender systems - Neural Collaborative Filtering
> Demo

- toc: true 
- badges: true
- comments: true
- hide: true
- categories: [demo, neural networks, deep learning, recommender systems, paper]

Download dependencies and run `tensorboard` in the background:


In [3]:
"""
!pip install tensorflow lightfm pandas

%load_ext tensorboard
!tensorboard --logdir 2020-09-11-neural_collaborative_filter/logs &
"""

'\n!pip install tensorflow lightfm pandas\n\n%load_ext tensorboard\n!tensorboard --logdir 2020-09-11-neural_collaborative_filter/logs &\n'

In [4]:
#hide
import datetime
import os

import lightfm
import numpy as np
import pandas as pd
import tensorflow as tf
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from scipy import sparse
from tensorboard import notebook

In [5]:
#hide
print(f"Tensorflow version: {tf.__version__}")
print(f"LightFM version: {lightfm.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

Tensorflow version: 2.5.0-dev20210122
LightFM version: 1.16
Pandas version: 1.1.4
Numpy version: 1.19.2


In [36]:
TOP_K = 5
N_EPOCHS = 10

# Data

![](https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/matrix_factorization_with_alpha.png "Credit: https://developers.google.com/machine-learning/recommendation/collaborative/basics")

In [7]:
#hide_input
data = fetch_movielens(min_rating=3.0)

print("Interaction matrix:")
print(data["train"].toarray()[:10, :10])

Interaction matrix:
[[5 3 4 3 3 5 4 0 5 3]
 [4 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [4 0 0 0 0 0 0 4 4 0]
 [0 0 0 5 0 0 5 5 5 4]
 [0 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 4 0 0 0]
 [4 0 0 4 0 0 0 0 4 0]]


In [8]:
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 74627 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 7893 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}

In [9]:
#collapse
for dataset in ["test", "train"]:
    data[dataset] = (data[dataset].toarray() > 0).astype("int8")  # astype("int8")을 통해 조건값 이진화.
    
# Make the ratings binary
print("Interaction matrix:")
print(data["train"][:10, :10])

print("\nRatings:")
unique_ratings = np.unique(data["train"])
print(unique_ratings)

Interaction matrix:
[[1 1 1 1 1 1 1 0 1 1]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 1 1 1 1]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 0 1 0]]

Ratings:
[0 1]


In [10]:
from typing import List

def wide_to_long(wide: np.array, possible_ratings: List[int]) -> np.array:
    """Go from wide table to long.
    :param wide: wide array with user-item interactions
    :param possible_ratings: list of possible ratings that we may have."""
    
    def _get_ratings(arr: np.array, rating: int) -> np.array:
        """Generate long array for the rating provided
        :param arr: wide array with user-item interactions
        :param rating: the rating that we are interested"""
        idx = np.where(arr == rating)
        return np.vstack(
            (idx[0], idx[1], np.ones(idx[0].size, dtype="int8") * rating)  # 
        ).T  # .T: 행과열 전치
    
    long_arrays = []
    for r in possible_ratings:
        long_arrays.append(_get_ratings(wide, r))
        
    return np.vstack(long_arrays)

In [11]:
long_train = wide_to_long(data["train"], unique_ratings)
df_train = pd.DataFrame(long_train, columns=['user_id', 'item_id', 'interaction'])

In [12]:
long_train

array([[   0,    7,    0],
       [   0,   10,    0],
       [   0,   19,    0],
       ...,
       [ 942, 1187,    1],
       [ 942, 1227,    1],
       [ 942, 1329,    1]])

In [13]:
#hide_input
print("All interactions:")
df_train

All interactions:


,user_id,item_id,interaction
0,0,7,0
1,0,10,0
2,0,19,0
3,0,20,0
4,0,26,0
...,...,...,...
1586121,942,1043,1
1586122,942,1073,1
1586123,942,1187,1
1586124,942,1227,1


In [14]:
#hide_input
print("Only positive interactions:")
df_train[df_train["interaction"] > 0].head()

Only positive interactions:


,user_id,item_id,interaction
1511499,0,0,1
1511500,0,1,1
1511501,0,2,1
1511502,0,3,1
1511503,0,4,1


# The model (Neural Collaborative Filtering)

<center><img src="https://raw.githubusercontent.com/murilo-cunha/inteligencia-superficial/master/images/2020-09-11-neural_collaborative_filter/ncf_all_with_alpha.png" width="70%" url="https://developers.google.com/machine-learning/recommendation/collaborative/basics" description="Fonte: https://developers.google.com/machine-learning/recommendation/collaborative/basics" /> </center>

In [29]:
import tensorflow.keras as keras
from tensorflow.keras.layers import (
    Concatenate,
    Dense,
    Embedding,
    Flatten,
    Input,
    Multiply,
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2


def create_ncf(
    number_of_users: int,
    number_of_items: int,
    latent_dim_mf: int = 4,
    latent_dim_mlp: int = 32,
    reg_mf: int = 0,
    reg_mlp: int = 0.01,
    dense_layers: List[int] = [8, 4],
    reg_layers: List[int] = [0.01, 0.01],
    activation_dense: str = 'relu',
) -> keras.Model:
    
    # input layer
    user = Input(shape=(), dtype='int32', name='user_id')
    item = Input(shape=(), dtype='int32', name='item_id')
    
    # embedding layers
    mf_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mf,
        name="mf_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )
    mf_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mf,
        name="mf_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mf),
        input_length=1,
    )
    
    mlp_user_embedding = Embedding(
        input_dim=number_of_users,
        output_dim=latent_dim_mlp,
        name="mlp_user_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )
    
    mlp_item_embedding = Embedding(
        input_dim=number_of_items,
        output_dim=latent_dim_mlp,
        name="mlp_item_embedding",
        embeddings_initializer="RandomNormal",
        embeddings_regularizer=l2(reg_mlp),
        input_length=1,
    )
    
    # MF vector
    mf_user_latent = Flatten()(mf_user_embedding(user))
    mf_item_latent = Flatten()(mf_item_embedding(item))
    mf_cat_latent = Multiply()([mf_user_latent, mf_item_latent])

    # MLP vector
    mlp_user_latent = Flatten()(mlp_user_embedding(user))
    mlp_item_latent = Flatten()(mlp_item_embedding(item))
    mlp_cat_latent = Concatenate()([mlp_user_latent, mlp_item_latent])
    
    mlp_vector = mlp_cat_latent
    
    # build dense layers for model
    for i in range(len(dense_layers)):
        layer = Dense(
            dense_layers[i],
            activity_regularizer=l2(reg_layers[i]),
            activation=activation_dense,
            name="layer%d" % i,
        )
        mlp_vector = layer(mlp_vector)
        
    predict_layer = Concatenate()([mf_cat_latent, mlp_vector])
    
    result = Dense(
        1, activation="sigmoid", kernel_initializer="lecun_uniform", name="interaction"
    )
    
    output = result(predict_layer)
    
    model = Model(
        inputs=[user, item],
        outputs=[output],
    )
    
    return model

In [30]:
# collapse
from tensorflow.keras.optimizers import Adam

n_users, n_items = data["train"].shape
ncf_model = create_ncf(n_users, n_items)

ncf_model.compile(
    optimizer=Adam(),
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="acu"),
    ],
)
ncf_model._name = "neural_collaborative_filtering"
ncf_model.summary()

Model: "neural_collaborative_filtering"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
item_id (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
mlp_user_embedding (Embedding)  (None, 32)           30176       user_id[0][0]                    
__________________________________________________________________________________________________
mlp_item_embedding (Embedding)  (None, 32)           53824       item_id[0][0]                    
_____________________________________________________________________

In [33]:
def make_tf_dataset(
    df: pd.DataFrame,
    targets: List[str],
    val_split: float = 0.1,
    batch_size: int = 512,
    seed=42,
):
    """Make TensorFlow dataset from Pandas DataFrame.
    :param df: input DataFrame - only contains features and target(s)
    :param targets: list of columns names corresponding to targets
    :param val_split: fraction of the data that should be used for validation
    :param batch_size: batch size for training
    :param seed: random seed for shuffling the data - setting to `None` will not shuffle the data"""
    
    n_val = round(df.shape[0] * val_split)
    if seed:
        # shuffle all the rows
        x = df.sample(frac=1, random_state=seed).to_dict("series")
    else:
        x = df.to_dict("series")
    y = dict()
    for t in targets:
        y[t] = x.pop(t)
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    
    ds_val = ds.take(n_val).batch(batch_size)
    ds_train = ds.skip(n_val).batch(batch_size)
    return ds_train, ds_val

In [34]:
# create train and validation datasets
ds_train, ds_val = make_tf_dataset(df_train, ["interaction"])

In [38]:
%%time
# define logs and callbacks
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=0
)

train_hist = ncf_model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=N_EPOCHS,
    callbacks=[tensorboard_callback, early_stopping_callback],
    verbose=1,
)

Epoch 1/10
2789/2789 [==============================] - 14s 5ms/step - loss: 0.1244 - tp: 5683.0000 - fp: 2214.0000 - tn: 1358200.0000 - fn: 61416.0000 - accuracy: 0.9554 - precision: 0.7196 - recall: 0.0847 - acu: 0.9159 - val_loss: 0.1185 - val_tp: 1247.0000 - val_fp: 525.0000 - val_tn: 150560.0000 - val_fn: 6281.0000 - val_accuracy: 0.9571 - val_precision: 0.7037 - val_recall: 0.1656 - val_acu: 0.9210
CPU times: user 30.4 s, sys: 2.46 s, total: 32.9 s
Wall time: 13.8 s


In [39]:
long_test = wide_to_long(data["train"], unique_ratings)
df_test = pd.DataFrame(long_test, columns=["user_id", "item_id", "interaction"])
ds_test, _ = make_tf_dataset(df_test, ["interaction"], val_split=0, seed=None)

In [40]:
%%time
ncf_predictions = ncf_model.predict(ds_test)
df_test["ncf_predictions"] = ncf_predictions

CPU times: user 6.28 s, sys: 306 ms, total: 6.59 s
Wall time: 5.22 s


In [41]:
#hide_input
df_test.head()

,user_id,item_id,interaction,ncf_predictions
0,0,7,0,0.483398
1,0,10,0,0.468876
2,0,19,0,0.115457
3,0,20,0,0.125516
4,0,26,0,0.106007


In [45]:
#hide
# sanity checks - stop execution if we have low standard deviation (all recommendations are the same)
std = df_test.describe().loc["std", "ncf_predictions"]
if std < 0.01:
    raise ValueError("Model predictions have standard deviation of less than le-2.")

In [46]:
#collapse
data["ncf_predictions"] = df_test.pivot(
    index="user_id", columns="item_id", values="ncf_predictions"
).values
print("Neural collaborative filtering predictions")
print(data["ncf_predictions"][:10, :4])

Neural collaborative filtering predictions
[[0.6269704  0.24987572 0.1468502  0.49421766]
 [0.2256749  0.02604404 0.02385423 0.0362916 ]
 [0.19684681 0.03824714 0.03633711 0.03165561]
 [0.1689327  0.02127731 0.01980689 0.01723003]
 [0.56245416 0.2204996  0.1208525  0.31617355]
 [0.6136544  0.12170559 0.07536039 0.46111798]
 [0.6823194  0.37174204 0.23828226 0.6701757 ]
 [0.4547332  0.05517426 0.02830127 0.10933998]
 [0.19163734 0.01153988 0.00932038 0.01495835]
 [0.56063217 0.10945806 0.07458898 0.39636242]]


In [47]:
precision_ncf = tf.keras.metrics.Precision(top_k=TOP_K)
recall_ncf = tf.keras.metrics.Recall(top_k=TOP_K)

precision_ncf.update_state(data["test"], data["ncf_predictions"])
recall_ncf.update_state(data["test"], data["ncf_predictions"])
print(
    f"At K = {TOP_K}, we have a precision of {precision_ncf.result().numpy():.5f} and a recall of {recall_ncf.result().numpy():.5f}"
)

At K = 5, we have a precision of 0.09841 and a recall of 0.05879


In [49]:
%%time
# LightFM model
norm = lambda x: (x - np.min(x)) / np.ptp(x)
lightfm_model = LightFM(loss="warp")
lightfm_model.fit(sparse.coo_matrix(data["train"]), epochs=N_EPOCHS)

lightfm_predictions = lightfm_model.predict(
    df_test["user_id"].values, df_test["item_id"].values
)
df_test["lightfm_predictions"] = lightfm_predictions
wide_predictions = df_test.pivot(
    index="user_id", columns="item_id", values="lightfm_predictions"
).values
data["lightfm_predictions"] = norm(wide_predictions)

# compute the metrics
precision_lightfm = tf.keras.metrics.Precision(top_k=TOP_K)
recall_lightfm = tf.keras.metrics.Recall(top_k=TOP_K)
precision_lightfm.update_state(data["test"], data["lightfm_predictions"])
recall_lightfm.update_state(data["test"], data["lightfm_predictions"])
print(
    f"At K = {TOP_K}, we have a precision of {precision_lightfm.result().numpy():.5f} and a recall of {recall_lightfm.result().numpy():.5f}"
)

At K = 5, we have a precision of 0.10880 and a recall of 0.06499
CPU times: user 853 ms, sys: 45.9 ms, total: 899 ms
Wall time: 784 ms
